In [1]:
import urllib
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import re

In [2]:
def get_url(url):
    
    page = urllib.request.urlopen(url)

#Cooking the Soup
    soup = BeautifulSoup(page,"html.parser")

    web_list=[]
#Scraping "Link" (href)
    for a in soup.find_all("a", href=True):
        web_list.append(a["href"])
    speech=[]
    for i in web_list:
        if i.startswith("speeches"):
            speech.append(i)
    speech_web=[]
    for i in speech:
        if i.endswith('htm'):
            speech_web.append(i)
    
    return speech_web

def modify_url(speech_web):
    start = speech_web.index('speeches/barackobama/barackobamainauguraladdress.htm')
    speech_president = speech_web[start:]

# remove replicate
    new = []
    for i in speech_president:
        if i not in new:
            new.append(i)

    speech_url = []
    for val in range(len(new)):
        speech_url.append("https://www.americanrhetoric.com/" + str(new[val])) 
   
    return speech_url

In [3]:
def extract_text(url):
        user = {
            'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"
        }
        resp = requests.get(url, headers=user)
        soup = BeautifulSoup(resp.text, "html")
       
        content=soup.find_all("font",face="Verdana", size="2")
        l=[]
        speech={}

        speech['url']=url
        for i in range(len(content)):
            l.append(content[i].text.replace('\r\n\t\t', ' ').strip())
        speech['content']=''
        for x in l:
            x=x.replace("\'", ' ').strip()
            speech['content']+=" "+x
        return speech



In [4]:
def extract_title(url):
    user = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"}
    resp = requests.get(url, headers=user)
    soup = BeautifulSoup(resp.text, "lxml")
        
    title=soup.find_all('font', color="#000048")
    title_list=[]
    for i in range(len(title)):
            title_list.append(title[i].text)
    return title_list

In [5]:
def extract_date(url):
        user = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"}
        resp = requests.get(url, headers=user)
        soup = BeautifulSoup(resp.text, "lxml")
        
        date=soup.find_all('font', size="4", face="Tahoma")
        date_list=[]
        for i in range(len(date)):
            date_list.append(date[i].text)
        return date_list
        

In [6]:

url = "https://www.americanrhetoric.com/barackobamaspeeches.htm"
speech_web=get_url(url)
speech_url=modify_url(speech_web)
    
date=extract_date(url)
date_list=date[date.index("20 Jan 2009"):]
title=extract_title(url)
for i in date[:]:
    if i in title:
        title.remove(i)
title=list(filter(lambda a: a != 'O', title))
title=list(filter(lambda a: a != 'ç', title))
title=list(filter(lambda a: a != 'ois \r\n\t\t\tHollande', title))
title_list=title[title.index('FIRST PRESIDENTIAL INAUGURAL ADDRESS'):]
    

data_list=[]
for i in range(len(speech_url)):
    data_list.append(extract_text(speech_url[i]))
df=pd.DataFrame(data_list)
df['title']=title_list
df['date']=date_list
    
df.to_csv('obama_speech.csv')

if __name__ == '__main__':
    main()